In [1]:
import numpy as np



In [2]:
# Define a toy protein alphabet and states
alphabet = "ACDEFGHIKLMNPQRSTVWY"  # 20 amino acids
states = ['M', 'I']  # Match and Insert states

# Define model parameters for our toy HMM:
# Initial state probabilities (pi)
pi = {
    'M': 0.9,
    'I': 0.1
}

# Transition probabilities: a[state_from][state_to]
A = {
    'M': {'M': 0.8, 'I': 0.2},
    'I': {'M': 0.4, 'I': 0.6}
}

# Emission probabilities: e[state][symbol]
# For simplicity, we assume each state emits a symbol from the alphabet.
# In a real model, these would be estimated from a multiple sequence alignment.
# Here we give arbitrary probabilities for demonstration.
def random_emission_probs():
    probs = np.random.rand(len(alphabet))
    return dict(zip(alphabet, probs / probs.sum()))

# For reproducibility, we can define fixed emission probabilities:
E = {
    'M': {aa: 1/len(alphabet) for aa in alphabet},  # uniform for match state
    'I': {aa: 1/len(alphabet) for aa in alphabet}   # uniform for insert state
}



In [8]:
print("states: ",states)

print("pi: ",pi)

print("A: ",A)

print("E: ",E)

states:  ['M', 'I']
pi:  {'M': 0.9, 'I': 0.1}
A:  {'M': {'M': 0.8, 'I': 0.2}, 'I': {'M': 0.4, 'I': 0.6}}
E:  {'M': {'A': 0.05, 'C': 0.05, 'D': 0.05, 'E': 0.05, 'F': 0.05, 'G': 0.05, 'H': 0.05, 'I': 0.05, 'K': 0.05, 'L': 0.05, 'M': 0.05, 'N': 0.05, 'P': 0.05, 'Q': 0.05, 'R': 0.05, 'S': 0.05, 'T': 0.05, 'V': 0.05, 'W': 0.05, 'Y': 0.05}, 'I': {'A': 0.05, 'C': 0.05, 'D': 0.05, 'E': 0.05, 'F': 0.05, 'G': 0.05, 'H': 0.05, 'I': 0.05, 'K': 0.05, 'L': 0.05, 'M': 0.05, 'N': 0.05, 'P': 0.05, 'Q': 0.05, 'R': 0.05, 'S': 0.05, 'T': 0.05, 'V': 0.05, 'W': 0.05, 'Y': 0.05}}


In [12]:
# Example protein sequence (as a string)
sequence = "ACDGHIK"

# ---------------------------
# Forward Algorithm
# ---------------------------
def forward_algorithm(seq, states, pi, A, E):
    T = len(seq)
    # Initialize alpha: a dictionary mapping state to probability at each position
    alpha = [{} for _ in range(T)]
    
    # Initialization: position 0
    for state in states:
        alpha[0][state] = pi[state] * E[state][seq[0]]
    
    # Recursion: positions 1 to T-1
    for t in range(1, T):
        for j in states:
            # sum over previous states
            alpha[t][j] = sum(alpha[t-1][i] * A[i][j] for i in states) * E[j][seq[t]]
    
    # Termination: sum over final states
    print(alpha)
    likelihood = sum(alpha[T-1][state] for state in states)
    return likelihood, alpha

# ---------------------------
# Viterbi Algorithm
# ---------------------------
def viterbi_algorithm(seq, states, pi, A, E):
    T = len(seq)
    # Initialize viterbi probability matrix and backpointer
    V = [{} for _ in range(T)]
    backpointer = [{} for _ in range(T)]
    
    # Initialization: position 0
    for state in states:
        V[0][state] = np.log(pi[state]) + np.log(E[state][seq[0]])
        backpointer[0][state] = None
    
    # Recursion: positions 1 to T-1
    for t in range(1, T):
        for j in states:
            max_prob, prev_st = max(
                ((V[t-1][i] + np.log(A[i][j]), i) for i in states),
                key=lambda x: x[0]
            )
            V[t][j] = max_prob + np.log(E[j][seq[t]])
            backpointer[t][j] = prev_st
    
    # Termination: find the highest probability in the last column
    last_state, max_log_prob = max(V[T-1].items(), key=lambda x: x[1])
    
    # Trace back the path
    best_path = [last_state]
    for t in range(T-1, 0, -1):
        best_path.insert(0, backpointer[t][best_path[0]])
    
    return best_path, max_log_prob



In [13]:
# ---------------------------
# Run the Algorithms
# ---------------------------
likelihood, alpha = forward_algorithm(sequence, states, pi, A, E)
print("Forward Algorithm:")
print("Sequence likelihood:", likelihood)

best_path, best_log_prob = viterbi_algorithm(sequence, states, pi, A, E)
print("\nViterbi Algorithm:")
print("Best state path:", best_path)
print("Log-probability of the path:", best_log_prob)

[{'M': 0.045000000000000005, 'I': 0.005000000000000001}, {'M': 0.0019000000000000004, 'I': 0.0006000000000000002}, {'M': 8.800000000000004e-05, 'I': 3.700000000000001e-05}, {'M': 4.260000000000002e-06, 'I': 1.990000000000001e-06}, {'M': 2.1020000000000012e-07, 'I': 1.0230000000000006e-07}, {'M': 1.0454000000000007e-08, 'I': 5.171000000000004e-09}, {'M': 5.215800000000005e-10, 'I': 2.596700000000002e-10}]
Forward Algorithm:
Sequence likelihood: 7.812500000000007e-10

Viterbi Algorithm:
Best state path: ['M', 'M', 'M', 'M', 'M', 'M', 'M']
Log-probability of the path: -22.414347738421018
